# 如何将参数从一个步骤传递到下一个步骤
:::info 前提条件
本指南假定读者熟悉以下概念：- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)- [链式运行](/docs/how_to/sequence/)- [并行调用可运行对象](/docs/how_to/parallel/)- [自定义函数](/docs/how_to/functions/)
:::

在构建包含多个步骤的链时，有时您会希望将前几步的数据原封不动地传递到后续步骤作为输入。[`RunnablePassthrough`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html)类正是为此设计，通常需要与[RunnableParallel](/docs/how_to/parallel/)配合使用，以便在构建的链中将数据透传到后续步骤。
请参考以下示例：

In [ ]:
%pip install -qU langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

如上所示，`passed`键被调用了`RunnablePassthrough()`，因此它直接传递了`{'num': 1}`。
我们还在映射中设置了第二个键 `modified`。这里使用了一个 lambda 表达式，将 `num` 加 1 后作为单一值设置，最终得到键 `modified` 的值为 `2`。

## 检索示例
在下面的示例中，我们将看到一个更贴近实际应用的场景：通过将`RunnablePassthrough`与`RunnableParallel`结合使用，构建一个链式流程来规范提示词输入的格式化处理：

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

此处提示词的输入预期为一个包含"context"和"question"键的映射。用户输入仅为问题部分。因此我们需要通过检索器获取上下文内容，并将用户输入以"question"键透传。`RunnablePassthrough`组件使我们能够将用户问题同时传递给提示词模板和模型。
## 后续步骤
现在你已经学会了如何通过链传递数据，以帮助格式化流经链的数据。
要了解更多信息，请参阅本节中关于可运行项的其他操作指南。